<a href="https://colab.research.google.com/github/Microflow-IO/Microflow-IO.github.io/blob/main/DeepSeek_RAG/RAG_DeepSeek_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive RAG with DeepSeek and LangChain

This notebook shows an easy RAG (Retrieval Augmented Generation) with DeepSeek model from Hugging Face [`deepseek-ai/DeepSeek-R1`](https://huggingface.co/deepseek-ai/DeepSeek-R1), and LangChain.


**RAG process**

The RAG (Retrieval-Augmented Generation) system combines a retrieval system with an LLM. The system first retrieves relevant documents from a corpus using a vector database, then uses an LLM hosted in Hugging Face to generate answers based on the retrieved documents.


# Prepare Workspace

In [21]:
!pip install -q torch transformers sentence-transformers faiss-cpu pypdf

In [ ]:
!pip install -U langchain-huggingface

In [3]:
!pip install -q langchain langchain-community

In [23]:
import langchain as lc
from langchain import LLMMathChain
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline


## Upload the data


In [24]:
# Load content from local PDFs
loader = PyPDFLoader("http://stu.jxit.net.cn:88/uniprobe/doc/uniprobe-help.pdf")
docs = loader.load()

In [ ]:
# Define the document:
Document(page_content="DeepSeek-R1:Incentivizing Reasoning Capability in LLMs via Reinforcement Learning.",
         metadata={
             'document_id' : '2501.12948v1',
             'document_source' : "ArXiv",
             'document_create_time' : "2025"
         })

In [ ]:
print("\nPage Content: ", docs[0].page_content)
print("\nMeta Data: ", docs[0].metadata)

In [28]:
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)
chunked_docs = splitter.split_documents(docs)
print(chunked_docs)

[Document(metadata={'source': 'http://stu.jxit.net.cn:88/uniprobe/doc/uniprobe-help.pdf', 'page': 0, 'page_label': '1'}, page_content='UniServer 1.0用户手册 \n \n \nUniServer  \n版本1.0 \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n  \n \n \n \n \n \n2021年4月07'), Document(metadata={'source': 'http://stu.jxit.net.cn:88/uniprobe/doc/uniprobe-help.pdf', 'page': 1, 'page_label': '2'}, page_content='UniServer 1.0用户手册 \n \n \n \n \nUniServer \n \n \n \n \n \n \n \n目录 \n第一章 ................................................................................................................................................. 1 \n体系结构方面的考虑 ........................................................................................................................... 1 \n1.1  最小设置 ............................................................................................................................... 1'), Document(metadata={'source': 'http://stu.jxit.net.cn:88/uniprobe/doc/

In [29]:
print("PDF Splited by Chunks - You have {0} number of chunks.".format(len(docs)))

PDF Splited by Chunks - You have 491 number of chunks.


## Embeddings + Retriever

For embeddings I use the `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5) embeddings model.

To create the vector database, I use `FAISS`, a library developed by Facebook AI. This library offers efficient similarity search and clustering of dense vectors.

In [30]:
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

In [31]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3}
)

## Load the model

In [32]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Setup the RAG

First, I create a text_generation pipeline using the loaded model and its tokenizer.

Next, I create a prompt template.

Then, I combined the `llm_chain` with the retriever to create a RAG chain.

In [33]:
# Pipeline for text generation
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Prompt template to match desired output format
prompt_template = """
You are a professional AI researcher, give an help in study. Use the following context to answer the question using information provided by the paper:

{context}

Question: {question}
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()


rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)


Device set to use cuda:0


# Questions

In [37]:
question = "如何通过uniserver web界面为探针分配配置？"
print(rag_chain)
# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

first={
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7912aa3d1bd0>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
} middle=[PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a professional AI researcher, give an help in study. Use the following context to answer the question using information provided by the paper:\n\n{context}\n\nQuestion: {question}\n'), HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x791294bef950>, model_id='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B')] last=StrOutputParser()
</think>

要通过Uniserver Web界面为探针分配配置，请按照以下步骤操作：

1. **打开Uniserver Web界面**：
   - 打开Uniserver Web界面，确保您已经登录并进入Uniserver的企业服务。

2. **访问探针配置管理页面**：
   - 在主页面上找到“企业/档案”模块。
   - 点击“探针”标签，进入探针配置管理页面。

3. **查看当前配置**：
   - 在探针配置管理页面，您可以看到当前的探针配置，包括探针名称、类型和相关参数。

4. **添加新探针或修改现有探

In [ ]:
question = "What is cold-start data and why is it used in DeepSeek-R1 training?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

</think>

Cold-start data refers to initial training data that is scarce or not available during the initial phase of a machine learning model's development. It is often used when there is limited data available to train a model effectively.

In the context of DeepSeek-R1 training:
- **Cold-start data** is utilized to initialize the model's knowledge base.
- The model starts with minimal information but gradually gains expertise through repeated training cycles.
- This approach helps the model adapt better to new challenges and improve its overall performance over time.

By leveraging cold-start data, DeepSeek-R1 can efficiently learn and refine its capabilities, making it more effective at solving complex problems.


In [ ]:
question = "What are DeepSeek-R1-Zero and DeepSeek-R1?"

# Invoke the chain to generate answers
result = rag_chain.invoke(question)

# Display the output
print(result)

</think>

DeepSeek-R1-Zero and DeepSeek-R1 are both models developed by DeepSeek, but they represent different versions or iterations of the same system.

- **DeepSeek-R1**:
  - **Description**: It is a language model developed by DeepSeek, designed for tasks such as text generation, question answering, and problem-solving.
  - **Key Features**:
    - **Language Model**: Utilizes transformer-based architectures to handle sequence modeling tasks.
    - **Inference Speed**: Achieves high inference speeds, making it suitable for real-world applications.
    - **Task Capabilities**: Supports a wide range of tasks including natural language understanding, generation, and reasoning.
    - **Performance**: Demonstrates strong performance across multiple domains, as evidenced by its results in benchmarks like ARXIV.

- **DeepSeek-R1-Zero**:
  - **Description**: This version introduces a "pure RL" approach, which likely refers to reinforcement learning techniques optimized for sequential decisi